Our goal is to upsample all classes with size < 1500 to at least 1500 samples.


First, we'll start by testing on a small sample of vectors, like our flattened test vectors.

Next, we'll create a python script that will run the upsample for our training and test sets.

Finally, we will rerun our model on the new upsampled data.

Shall we do a train, test, & validation set?

In [1]:
import os, pickle

import numpy as np
import pandas as pd


In [2]:
cd ../..

C:\Users\cryst\Documents\Work\DePaul-ResearchAssistant\vulnerability_detection


In [6]:
data_path = os.path.join('data')
sample_path = os.path.join(data_path, 'DLvectors', 'test_162classes_flattened', 'balanced_test.pkl')


In [7]:
with open(sample_path, 'rb') as f:
    data = pickle.load(f)
    
data

[[array([-0.01077731,  0.00788512,  0.00055634, ...,  0.        ,
          0.        ,  0.        ]),
  array([-0.01077731,  0.00788512,  0.00055634, ...,  0.01126998,
         -0.01393917, -0.00336132]),
  array([ 0.01459869, -0.01056556, -0.00759643, ...,  0.01532959,
         -0.01093371, -0.00962706]),
  array([-0.01077731,  0.00788512,  0.00055634, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.00650835,  0.00110991, -0.00233717, ...,  0.00392389,
         -0.00468164,  0.01028275]),
  array([ 0.00650835,  0.00110991, -0.00233717, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.00646882,  0.00886718,  0.01280721, ..., -0.00625886,
          0.00637691,  0.01208511]),
  array([ 0.01581487,  0.01038325, -0.00938146, ...,  0.0066965 ,
          0.01023589,  0.00654659]),
  array([-0.00894722,  0.01505855, -0.00351591, ...,  0.        ,
          0.        ,  0.        ]),
  array([ 0.01459869, -0.01056556, -0.00759643, ...,  0.00542845,
       

**The structure of our dataset:**

[
 [vectors]
 [labels]
 [vul line num]
 [filename]
 [type]
 [CWE-ID]
 [our testcase ID]
]

<br>
<br>

**Let's see how many samples of each class we have in this set...**

In [8]:
np.unique(data[-2], return_counts=1)

(array([ 15,  20,  23,  36,  59,  78,  79,  80,  88,  89,  90,  94,  99,
        114, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129, 134, 170,
        176, 189, 190, 191, 193, 194, 195, 196, 197, 200, 223, 226, 242,
        244, 252, 253, 254, 255, 256, 259, 269, 272, 284, 310, 319, 321,
        325, 327, 328, 347, 362, 363, 364, 366, 367, 369, 377, 390, 398,
        399, 400, 401, 404, 412, 414, 415, 416, 426, 427, 457, 459, 464,
        467, 468, 469, 475, 476, 478, 479, 506, 510, 535, 543, 588, 590,
        591, 605, 606, 609, 617, 663, 665, 666, 667, 674, 675, 676, 680,
        681, 682, 685, 688, 690, 704, 758, 761, 762, 764, 765, 771, 772,
        773, 774, 775, 780, 785, 787, 789, 805, 806, 820, 821, 822, 824,
        828, 831, 832, 833, 834, 835, 839, 843, 909]),
 array([ 6, 39, 39, 39,  9, 39, 25, 39, 39, 39, 39, 39,  1, 39, 39, 39, 39,
        39,  2, 39, 39, 39, 39, 39, 39, 39,  2, 39, 39, 39,  1, 39, 39, 39,
        25, 39,  1, 10,  1, 14, 14, 14, 29,  5, 37,  9,  5,  2,

**Let's try upsampling all samples to 39**

In [28]:
test_set = [np.array([[-7.77,  7.77,  7.77], [-7.07, 7.07, 7.07], [-1.01,  1.01,  1.11], [5.55, -5.55, 5.45]]), [0,1,0, 0], [44, 6,999, 8], ['testa', 'testb', 'testc', 'testd'], ['API', 'API','PTR', 'AE'],[119, 119, 120, 704], [888, 999, 20, 70]]
test_set

[array([[-7.77,  7.77,  7.77],
        [-7.07,  7.07,  7.07],
        [-1.01,  1.01,  1.11],
        [ 5.55, -5.55,  5.45]]),
 [0, 1, 0, 0],
 [44, 6, 999, 8],
 ['testa', 'testb', 'testc', 'testd'],
 ['API', 'API', 'PTR', 'AE'],
 [119, 119, 120, 704],
 [888, 999, 20, 70]]

Get ratios

In [29]:
# return the indices of the unique array that can be used to reconstruct the array.
classes, y_indices = np.unique(test_set[-2], return_inverse=True)
print(classes)
print(y_indices)

[119 120 704]
[0 0 1 2]


In [30]:
class_counts = np.bincount(y_indices)
class_counts

array([2, 1, 1], dtype=int64)

In [31]:
# Find the sorted list of instances for each class
class_indices = np.split(np.argsort(y_indices, kind='mergesort'),
                        np.cumsum(class_counts)[:-1])
print(f'Classes: {classes}.\n.\nClass Counts:{class_counts}.\n')

Classes: [119 120 704].
.
Class Counts:[2 1 1].



In [32]:
class_indices

[array([0, 1], dtype=int64), array([2], dtype=int64), array([3], dtype=int64)]

In [33]:
# turn each row into an numpy array so we can easily create subsets through indexing
vectors = np.array(test_set[0])
labels = np.array(test_set[1])
vul_line_num = np.array(test_set[2])
filename = np.array(test_set[3])
types = np.array(test_set[4])
cwes = np.array(test_set[5])
testcase_id = np.array(test_set[6])

new_set = [ vectors, labels, vul_line_num, filename, types, cwes, testcase_id ]
new_set

[array([[-7.77,  7.77,  7.77],
        [-7.07,  7.07,  7.07],
        [-1.01,  1.01,  1.11],
        [ 5.55, -5.55,  5.45]]),
 array([0, 1, 0, 0]),
 array([ 44,   6, 999,   8]),
 array(['testa', 'testb', 'testc', 'testd'], dtype='<U5'),
 array(['API', 'API', 'PTR', 'AE'], dtype='<U3'),
 array([119, 119, 120, 704]),
 array([888, 999,  20,  70])]

Since the numpy functions to upsample didn't work, we'll do it the classic simple python way

In [34]:
final_set = [[], [], [], [], [],[], []]

# run on classes with < 2 samples
max_size = 3
for count, indices in zip(class_counts, class_indices):    
    if count < max_size:
        # repeat samples until we reach our target count
        # walk the samples for each data row
        for i in range(len(new_set)):
            # subset of the feature for given class
            data_row = new_set[i][indices]
            if type(data_row) == np.ndarray:
                data_row = list(data_row)
            
            # walk the array and append as we go along
            n_remaining = max_size - count
            length = len(data_row)
            for j in range(n_remaining):
                # make sure we can keep going even if we run out
                k = j % length
                data_row.append(data_row[k])
            final_set[i] += data_row
    else:
        for i in range(len(new_set)):
            data_row = list(new_set[i][indices])
            final_set[i] += data_row
 
final_set

[[array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-1.01,  1.01,  1.11]),
  array([-1.01,  1.01,  1.11]),
  array([-1.01,  1.01,  1.11]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45])],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [44, 6, 44, 999, 999, 999, 8, 8, 8],
 ['testa',
  'testb',
  'testa',
  'testc',
  'testc',
  'testc',
  'testd',
  'testd',
  'testd'],
 ['API', 'API', 'API', 'PTR', 'PTR', 'PTR', 'AE', 'AE', 'AE'],
 [119, 119, 119, 120, 120, 120, 704, 704, 704],
 [888, 999, 888, 20, 20, 20, 70, 70, 70]]

In [35]:
new_set

[array([[-7.77,  7.77,  7.77],
        [-7.07,  7.07,  7.07],
        [-1.01,  1.01,  1.11],
        [ 5.55, -5.55,  5.45]]),
 array([0, 1, 0, 0]),
 array([ 44,   6, 999,   8]),
 array(['testa', 'testb', 'testc', 'testd'], dtype='<U5'),
 array(['API', 'API', 'PTR', 'AE'], dtype='<U3'),
 array([119, 119, 120, 704]),
 array([888, 999,  20,  70])]

In [36]:
for i in range(len(final_set)):
    np.random.seed(1099)
    np.random.shuffle(final_set[i])

final_set

[[array([ 5.55, -5.55,  5.45]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([ 5.55, -5.55,  5.45]),
  array([-7.77,  7.77,  7.77]),
  array([-1.01,  1.01,  1.11]),
  array([-1.01,  1.01,  1.11]),
  array([ 5.55, -5.55,  5.45]),
  array([-1.01,  1.01,  1.11])],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [8, 6, 44, 8, 44, 999, 999, 8, 999],
 ['testd',
  'testb',
  'testa',
  'testd',
  'testa',
  'testc',
  'testc',
  'testd',
  'testc'],
 ['AE', 'API', 'API', 'AE', 'API', 'PTR', 'PTR', 'AE', 'PTR'],
 [704, 119, 119, 704, 119, 120, 120, 704, 120],
 [70, 999, 888, 70, 888, 20, 20, 70, 20]]

###  run on all samples indivudally to weed out non-vulnerable samples


#### 1. Merge files or get a file that has both training & testing

In [48]:
new_set = [np.array([[-7.77,  7.77,  7.77], [-7.07, 7.07, 7.07], [-1.01,  1.01,  1.11], [5.55, -5.55, 5.45]]), [1,1,0, 1], [44, 6,999, 8], ['testa', 'testb', 'testc', 'testd'], ['API', 'API','PTR', 'AE'],[119, 119, 120, 704], [888, 999, 20, 70]]
new_set



[array([[-7.77,  7.77,  7.77],
        [-7.07,  7.07,  7.07],
        [-1.01,  1.01,  1.11],
        [ 5.55, -5.55,  5.45]]),
 [1, 1, 0, 1],
 [44, 6, 999, 8],
 ['testa', 'testb', 'testc', 'testd'],
 ['API', 'API', 'PTR', 'AE'],
 [119, 119, 120, 704],
 [888, 999, 20, 70]]

#### 2. gather all vulnerable samples

In [49]:
vul_set = [[], [], [], [], [],[], []]

# First, gather all vulnerable samples
for i in range(len(new_set[0])):
    if new_set[1][i] == 1:
        vul_set[0].append(new_set[0][i])
        vul_set[1].append(new_set[1][i])
        vul_set[2].append(new_set[2][i])
        vul_set[3].append(new_set[3][i])
        vul_set[4].append(new_set[4][i])
        vul_set[5].append(new_set[5][i])
        vul_set[6].append(new_set[6][i])
vul_set


[[array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([ 5.55, -5.55,  5.45])],
 [1, 1, 1],
 [44, 6, 8],
 ['testa', 'testb', 'testd'],
 ['API', 'API', 'AE'],
 [119, 119, 704],
 [888, 999, 70]]

In [59]:
# return the indices of the unique array that can be used to reconstruct the array.
classes, y_indices = np.unique(vul_set[-2], return_inverse=True)
class_counts = np.bincount(y_indices)
# Find the sorted list of instances for each class
class_indices = np.split(np.argsort(y_indices, kind='mergesort'),
                        np.cumsum(class_counts)[:-1])
print(f'Classes: {classes}.\nClass Counts:{class_counts}.\n')
print()


# return final set of vul upsampled samples 
final_set = [[], [], [], [], [],[], []]

max_size = 15
for count, indices in zip(class_counts, class_indices):
    if count < max_size:
        # repeat samples until we reach our target count
        n = 0
        while (n < max_size):
        #for class_i in indices:            
            # walk the array and append as we go along
#             for j in range(max_size):
                # make sure we can keep going even if we run out
            k = n % count
            final_set[0].append(vul_set[0][indices[k]])
            final_set[1].append(vul_set[1][indices[k]])
            final_set[2].append(vul_set[2][indices[k]])
            final_set[3].append(vul_set[3][indices[k]])
            final_set[4].append(vul_set[4][indices[k]])
            final_set[5].append(vul_set[5][indices[k]])
            final_set[6].append(vul_set[6][indices[k]])
            n += 1
    else:
        for class_i in indices:            
            # walk the array and append as we go along
            final_set[0].append(vul_set[0][class_i])
            final_set[1].append(vul_set[1][class_i])
            final_set[2].append(vul_set[2][class_i])
            final_set[3].append(vul_set[3][class_i])
            final_set[4].append(vul_set[4][class_i])
            final_set[5].append(vul_set[5][class_i])
            final_set[6].append(vul_set[6][class_i])
    

Classes: [119 704].
Class Counts:[2 1].




In [60]:
final_set

[[array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([-7.07,  7.07,  7.07]),
  array([-7.77,  7.77,  7.77]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45]),
  array([ 5.55, -5.55,  5.45])],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,

In [51]:
vectors = np.array(test_set[0])
labels = np.array(test_set[1])
vul_line_num = np.array(test_set[2])
filename = np.array(test_set[3])
types = np.array(test_set[4])
cwes = np.array(test_set[5])
testcase_id = np.array(test_set[6])

new_set = [ vectors, labels, vul_line_num, filename, types, cwes, testcase_id ]
new_set

            v = []
            l = []
            n = []
            f = []
            t = []
            c = []
            t = []

IndentationError: unexpected indent (Temp/ipykernel_83768/1957694891.py, line 12)